# Variations on sum

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.08953671978265532
 0.9229089118089155
 0.23243504664506265
 0.6371851471468601
 0.7169221441894971
 0.5793056825786933
 0.4838987204581333
 0.1222415129438692
 0.35036283046901895
 0.9835303277956495
 0.5532845489348261
 0.9949737622543187
 0.08664853082810353
 ⋮
 0.3108344772643048
 0.8904034531058518
 0.5831932569788159
 0.5927603341497338
 0.8244341030682507
 0.30322699998651537
 0.3878957340163689
 0.5296077850684455
 0.9335707183215762
 0.42470549327926177
 0.6087827717064473
 0.717840659068971

In [2]:
@time sum(a)
@time sum(a)
@time sum(a)

  0.023482 seconds (75.80 k allocations: 3.793 MiB)
  0.005327 seconds (5 allocations: 176 bytes)
  0.005942 seconds (5 allocations: 176 bytes)


4.998962299000235e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [3]:
using BenchmarkTools

#  1. Sum in C

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
c_sum(a)

4.998962299000687e6

In [6]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [7]:
c_sum(a) - sum(a)  

4.516914486885071e-7

In [8]:
≈  # alias for the `isapprox` function

isapprox (generic function with 8 methods)

In [9]:
?≈

"≈" can be typed by \approx<tab>

search: ≈



```
isapprox(x, y; rtol::Real=atol>0 ? 0 : √eps, atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to the usual `norm` function in LinearAlgebra, but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```


We can now benchmark the C code directly from Julia:

In [10]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.080 ms (0.00% GC)
  median time:      10.187 ms (0.00% GC)
  mean time:        10.268 ms (0.00% GC)
  maximum time:     15.446 ms (0.00% GC)
  --------------
  samples:          487
  evals/sample:     1

In [11]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 9.080271 msec


In [12]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 9.08027

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [13]:
using PyCall

In [14]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [15]:
pysum(a)

4.998962299000687e6

In [16]:
pysum(a) ≈ sum(a)

true

In [17]:
pysum(a) - sum(a)

4.516914486885071e-7

In [18]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  3
  --------------
  minimum time:     43.773 ms (0.00% GC)
  median time:      47.439 ms (0.00% GC)
  mean time:        47.979 ms (0.00% GC)
  maximum time:     63.981 ms (0.00% GC)
  --------------
  samples:          105
  evals/sample:     1

In [19]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 9.08027
  "Python built-in" => 43.7733

# 3. Python: `numpy` 

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [20]:
using Conda

In [21]:
np = pyimport("numpy")
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $(np.sum)($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  3
  --------------
  minimum time:     3.364 ms (0.00% GC)
  median time:      3.896 ms (0.00% GC)
  mean time:        4.099 ms (0.00% GC)
  maximum time:     10.147 ms (0.00% GC)
  --------------
  samples:          1217
  evals/sample:     1

In [22]:
numpy_sum(apy_list) # python thing

UndefVarError: UndefVarError: numpy_sum not defined

In [23]:
numpy_sum(apy_list) ≈ sum(a)

UndefVarError: UndefVarError: numpy_sum not defined

In [24]:
numpy_sum(apy_list) - sum(a)

UndefVarError: UndefVarError: numpy_sum not defined

In [25]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 9.08027
  "Python numpy"    => 3.3641
  "Python built-in" => 43.7733

# 4. Python, hand-written 

In [26]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x1393ee3b0>

In [27]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  3
  --------------
  minimum time:     199.998 ms (0.00% GC)
  median time:      210.346 ms (0.00% GC)
  mean time:        214.474 ms (0.00% GC)
  maximum time:     256.654 ms (0.00% GC)
  --------------
  samples:          24
  evals/sample:     1

In [28]:
@which sum([1.5])

sum(a::AbstractArray; dims) in Base at reducedim.jl:652

In [29]:
sum_py(apy_list)

4.998962299000687e6

In [30]:
sum_py(apy_list) ≈ sum(a)

true

In [31]:
sum_py(apy_list) - sum(a)

4.516914486885071e-7

In [32]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 9.08027
  "Python numpy"        => 3.3641
  "Python hand-written" => 199.998
  "Python built-in"     => 43.7733

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [33]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:652

In [34]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.601 ms (0.00% GC)
  median time:      4.217 ms (0.00% GC)
  mean time:        4.242 ms (0.00% GC)
  maximum time:     6.246 ms (0.00% GC)
  --------------
  samples:          1176
  evals/sample:     1

In [35]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 9.08027
  "Python numpy"        => 3.3641
  "Python hand-written" => 199.998
  "Python built-in"     => 43.7733
  "Julia built-in"      => 3.60092

# 6. Julia (hand-written) 

In [36]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [37]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.324 ms (0.00% GC)
  median time:      10.387 ms (0.00% GC)
  mean time:        10.620 ms (0.00% GC)
  maximum time:     17.604 ms (0.00% GC)
  --------------
  samples:          471
  evals/sample:     1

In [38]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 9.08027
  "Python numpy"        => 3.3641
  "Julia hand-written"  => 9.32369
  "Python hand-written" => 199.998
  "Python built-in"     => 43.7733
  "Julia built-in"      => 3.60092

# 7. Julia (hand-written + simd) 

In [39]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [40]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.633 ms (0.00% GC)
  median time:      4.121 ms (0.00% GC)
  mean time:        4.171 ms (0.00% GC)
  maximum time:     6.972 ms (0.00% GC)
  --------------
  samples:          1196
  evals/sample:     1

In [41]:
mysum_simd(a)

4.998962299000221e6

In [42]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any,Any} with 7 entries:
  "Julia hand-written simd" => 3.63319
  "C"                       => 9.08027
  "Python numpy"            => 3.3641
  "Julia hand-written"      => 9.32369
  "Python hand-written"     => 199.998
  "Python built-in"         => 43.7733
  "Julia built-in"          => 3.60092

# Summary

In [43]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 23, " "), lpad(round(value, digits=1), 6, " "))
end

Python numpy              3.4
Julia built-in            3.6
Julia hand-written simd   3.6
C                         9.1
Julia hand-written        9.3
Python built-in          43.8
Python hand-written     200.0
